In [5]:
!pip install pandas
!pip install gender-guesser

In [6]:
import pandas as pd
import numpy as np
import gender_guesser.detector as gender
import random

#### Load Datasets

In [7]:
baby_names = pd.read_csv("../data/Popular_Baby_Names_20240424.csv")
expense_budget = pd.read_csv("../data/Expense_Budget_20240424.csv", low_memory=False)
payroll = pd.read_csv("../data/Citywide_Payroll_Data__Fiscal_Year__20240424.csv", low_memory=False)

#### Data Cleaning

##### Payroll

In [8]:
payroll.isnull().any(axis=0)

Fiscal Year                   False
Payroll Number                 True
Agency Name                   False
Last Name                      True
First Name                     True
Mid Init                       True
Agency Start Date              True
Work Location Borough          True
Title Description              True
Leave Status as of June 30    False
Base Salary                   False
Pay Basis                     False
Regular Hours                 False
Regular Gross Paid            False
OT Hours                      False
Total OT Paid                 False
Total Other Pay               False
dtype: bool

Filter out all the rows that are missing names. We can't do gender analysis on them. There are 13,245 rows with missing names. 

In [9]:
payroll = payroll[payroll["First Name"].notna()]

Filter out all the rows that are missing start dates. There are only 63 rows with missing start dates. 

In [10]:
payroll = payroll[payroll["Agency Start Date"].notna()]


There are 506,233 rows with missing Work Location Borough. Since the number of missing names is large, we shouldn't just get rid of them. Some people might work in many different buroughs due to the nature of their jobs. Since this is a categorical value, we can have an extra value "Unknown" for any one with missing burough information. 

In [11]:
payroll["Work Location Borough"] = payroll["Work Location Borough"].fillna("UNKNOWN")

In [12]:
payroll['Agency Name'].value_counts()

Agency Name
DEPT OF ED PEDAGOGICAL            1099217
DEPT OF ED PER SESSION TEACHER     910621
POLICE DEPARTMENT                  536182
DEPT OF ED PARA PROFESSIONALS      371390
BOARD OF ELECTION POLL WORKERS     344458
                                   ...   
BRONX COMMUNITY BOARD #12              26
DISTRICTING COMMISSION                 24
BRONX COMMUNITY BOARD #1               21
BRONX COMMUNITY BOARD #3               19
BOARD OF CORRECTIONS                   17
Name: count, Length: 167, dtype: int64

##### Expense Budget

In [13]:
expense_budget["Agency Name"].values

array(['MAYORALTY', 'MAYORALTY', 'MAYORALTY', ...,
       'PUBLIC ADMINISTRATOR-RICHMOND COUNTY',
       'PUBLIC ADMINISTRATOR-RICHMOND COUNTY',
       'PUBLIC ADMINISTRATOR-RICHMOND COUNTY'], dtype=object)

#### Getting Gender Information

Checking overlap between baby names and payroll data

In [14]:
unique_baby = baby_names[baby_names["Child's First Name"].notnull()]
unique_payroll = payroll[payroll["First Name"].notnull()]

unique_baby = baby_names["Child's First Name"].unique()
unique_payroll = payroll["First Name"].unique()

unique_payroll = unique_payroll.astype(str)
unique_baby = unique_baby.astype(str)

unique_baby = np.char.upper(unique_baby)
unique_payroll = np.char.upper(unique_payroll)

len(np.intersect1d(unique_baby, unique_payroll))

1804

Use a gender prediction model

In [15]:
genModel = gender.Detector()

#run through model
def get_gender(name):
    return genModel.get_gender(name.capitalize())

payroll['Gender'] = payroll['First Name'].apply(get_gender)

#handle cases where the model is unsure and assign a gender
def finalize_gender(gender):
    if gender == "mostly_female":
        return "female"
    elif gender == "mostly_male":
        return "male"
    elif gender == "male" or gender == "female":
        return gender
    else:
        return random.choice(["male", "female"])
    
payroll['Gender'] = payroll['Gender'].apply(finalize_gender)
    

In [16]:
payroll['Gender'].value_counts()

Gender
female    3247725
male      2401743
Name: count, dtype: int64

#### Output the data in a json file

In [17]:
payroll.to_csv("../data/output/payroll.csv")

In [18]:
small = payroll.sample(n=50000)
small.to_csv("../data/output/small.csv")